<img src='otus.png'>

# Лекция 23: "Нейронные сети, часть 2"

План лекции:

1. Повторение предыдущей лекции.
  * Pytorch, вычислительные графы.
  * Нейронные сети: базовые определения.
  * Обучение нейронных сетей.
  * Как Pytorch упрощает работу с НС?
2. Различные алгоритмы градиентного спуска.
3. Пример с MNIST: data loader, data iterator. Полносвязная сеть.
4. Зачем нужен Deep Learning ("Глубокое обучение")?
5. Пример с MNIST. Сверточная сеть.
6. Прочие примеры DL успехов.

Если у вас появился существенный для понимания вопрос, пишите "111" в чат.

In [48]:
%matplotlib inline

## 1. Повторение предыдущей лекции.

### 1а. Pytorch, вычислительные графы.

Pytorch - фреймворк для работы с вычислительными графами. Чаще всего Pytorch называют фреймворком для "глубокого обучения", но это слегка сужает область его применения. 

Пример вычислительного графа для функции $x^2 + xy + (x + y)^2$:

<img src='graph.png'>

В прошлый раз мы видели:

**Как осуществлять операции с тензорами (n-мерные массивы) в Pytorch**

In [49]:
import torch
import numpy as np

a_numpy = np.random.randn(100, 1000)   # numpy array
a = torch.tensor(a_numpy)   # torch tensor / can be uploaded in GPU memory for faster computations

In [3]:
b = a ** 2 # element-wise operation

In [4]:
%timeit c = a_numpy.dot(a_numpy.T) # matrix multiplication with itself -- numpy

982 µs ± 125 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%timeit c = a.mm(a.t())   # matrix multiplication with itself -- pytorch

598 µs ± 99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Как рассчитывать градиенты для произвольных (дифференцируемых) функций**

In [50]:
x = torch.tensor(11, requires_grad=True, dtype=torch.float32)

In [51]:
print(x.grad)  # no gradients yet

None


In [59]:
y = x.pow(2)
print(y)

tensor(121., grad_fn=<PowBackward0>)


In [60]:
y.backward()
print(x.grad)

tensor(88.)


**Как пользоваться интерфейсом модуля torch.nn для быстрого создания НС.**

In [61]:
import torch.nn as nn

In [62]:
model = nn.Sequential(nn.Linear(20, 200),
                      nn.ReLU(),
                      nn.Linear(200, 1),
                      nn.Sigmoid()
                     )

In [63]:
inp = torch.randn(3, 20)
model(inp)

tensor([[0.5212],
        [0.4330],
        [0.5781]], grad_fn=<SigmoidBackward>)

### 1б. Нейронные сети, базовые определения.

Перцептрон - один нейрон, который берет взвешенную сумму входных признаков и применяет нелинейную функцию активации. Если функция активации это сигмоид, получаем логистическую регрессию.

<img src='neuron.png' width="30%" height="30%">

Пример полносвязной нейронной сети (в матричных обозначениях):

$$L_1 = f_1(W_1 A+b_1)$$
$$L_2 = f_2(W_2 L_1+b_2)$$
$$OUT = f_3(W_3 L_2+b_3)$$

https://goo.gl/LvPGqU

### 1в. Обучение НС (backpropagation). 
*На примере полносвязной сети, $k$ - номер слоя, $L$ - финальный слой*.

"Forward pass". Всего 2 уравнения:

1. $z^k = \omega^k a^{k-1} + b^k$  (weighted inputs)

2. $a^k = \sigma (z^k)$  (activations)

"Backward pass". Всего 4 уравнения:

deltas: $\delta^k = \partial Loss / \partial z^k$

1. $\delta^L = \partial Loss / \partial a^L $
2. $\delta^k = ((\omega^{k+1})^T \delta^{k+1}) \cdot \sigma' (z^k)$
3. $\partial Loss / \partial b^k = \delta^k$
4. $\partial Loss / \partial \omega^k = a^{k-1}\delta^k$

Здесь $\cdot$ это поэлементное умножение.

### 1г. Как Pytorch упрощает работу с НС.

Мы видели следующие полезные абстракции:

1. Модуль torch.nn
2. Оптимизаторы torch.optim
3. Функции потерь, например torch.nn.BCELoss()

На этой лекции:

1. Разберем torch.optim более подробно
2. Рассмотрим эффективные методы для организации датасетов

## 2. Различные алгоритмы градиентного спуска

Работаем в фреймворке "оптимизация эмпирического риска" (Empirical Risk Minimization, ERM).

**Дана обучающая выборка (задача классификации):**

$X =\{x_i\}_{i=1}^N$; $x_i \in \mathcal{R}^d$

$Y =\{y_i\}_{i=1}^N$; $y_i \in \mathcal{S}$, где $\mathcal{S}$ это набор возможных классов


**Минимизируем функцию потерь:**

$L(X, Y, f) = \frac{1}{N}\sum_{i=1}^{N}V(f(x_i), y_i)+\lambda R(f)$

Здесь $f_\theta(x)$ это та функция, которую мы ищем (в данной лекции это нейронная сеть) и $\theta$ - ее параметры. Слагаемое $R(f)$ - регуляризатор, $V$ - функция потерь.


**Как оптимизировать функцию $f$ по параметрам? Варианты:**

1. Случайный поиск <---- плохая идея
2. CEM (cross-entropy method) <---- более эффективно, но все еще медленно
2. Градиентный спуск <----- наш выбор (по крайней мере, сейчас..)

**Алгоритм градиентного спуска:**

$\theta_{t+1} = \theta_t - \alpha \nabla_\theta L(X, Y, f_{\theta_t})$


**Стохастический градиентный спуск:**

$L(X, Y, f) \rightarrow L_k(X, Y, f)$

$L_k(X, Y, f)$ - взято только k-ое слагаемое из общей функции потерь

Работа с минибатчами: берем не одно $k$-ое слагаемое, а сэмплим $n$ таких слагаемых случайным образом каждый раз во время градиентного спуска.

Интерпретация: мы приближаем истинное значение $L(X, Y, f)$ с помощью нескольких случайных значений.

Игрушечный пример: выборка из цифр 2 и 3, приближаем среднее мини-батчами.

**Разновидности градиентного спуска**

Рассмотрим только один вариант - "градиентный спуск с моментумом":

$v_{t+1} = \beta v_t + \nabla_\theta L(X, Y, f_{\theta_t})$

$\theta_{t+1} = \theta_t - \alpha v_{t+1}$

Аналогия: шар катится с горы и накапливает скорость.

**Отличная визуализация:** http://ruder.io/optimizing-gradient-descent/

**Более подробно про моментум:** https://distill.pub/2017/momentum/

Множество разновидностей градиентного спуска доступно в torch.optim

## 3. Пример с MNIST: data loader, data iterator.

In [64]:
import os
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms

In [65]:
BATCH_SIZE = 32
NUM_EPOCHS = 10
FOLDER = 'MNIST_data'

if not os.path.exists(FOLDER):
    os.mkdir(FOLDER)

In [66]:
trans = transforms.Compose([transforms.ToTensor()])

In [67]:
# данные
train_set = datasets.MNIST(root=FOLDER, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=FOLDER, train=False, transform=trans, download=True)

In [68]:
train_set

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: MNIST_data
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None

In [21]:
# итераторы
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=False)

In [22]:
sample_x, sample_y = next(iter(train_loader))
print(sample_x.size())
print(sample_y.size())

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [23]:
# один скрытый слой, на выходе 10 логитов (по числу классов)
model = torch.nn.Sequential(
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Linear(100, 10),    
)

In [24]:
opt = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [25]:
# процесс тренировки; тест напишите для упражнения
for epoch in range(NUM_EPOCHS):
    # trainning
    av_loss = 0.
    correct = 0.
    for x, y in tqdm.tqdm(train_loader):
        # рассчитываем функцию потерь
        x = x.view(BATCH_SIZE, -1)
        out = model(x)
        loss = criterion(out, y)
        # оптимизация параметров
        opt.zero_grad()
        loss.backward()
        opt.step()
        # подсчет статистики за эпоху
        pred = out.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        av_loss += loss.item()
    print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(train_loader.dataset) * 100))



  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 3/1875 [00:00<01:04, 28.90it/s]

  1%|          | 14/1875 [00:00<00:50, 37.02it/s]

  2%|▏         | 31/1875 [00:00<00:38, 48.17it/s]

  3%|▎         | 47/1875 [00:00<00:30, 60.93it/s]

  3%|▎         | 63/1875 [00:00<00:24, 74.54it/s]

  4%|▍         | 78/1875 [00:00<00:20, 87.43it/s]

  5%|▍         | 93/1875 [00:00<00:17, 99.03it/s]

  6%|▌         | 107/1875 [00:00<00:16, 107.90it/s]

  7%|▋         | 124/1875 [00:00<00:14, 120.29it/s]

  7%|▋         | 139/1875 [00:01<00:13, 125.43it/s]

  8%|▊         | 154/1875 [00:01<00:22, 76.34it/s] 

  9%|▉         | 166/1875 [00:01<00:20, 82.52it/s]

  9%|▉         | 177/1875 [00:01<00:19, 87.86it/s]

 10%|█         | 188/1875 [00:01<00:18, 91.01it/s]

 11%|█         | 199/1875 [00:01<00:18, 90.95it/s]

 11%|█         | 210/1875 [00:02<00:19, 87.36it/s]

 12%|█▏        | 225/1875 [00:02<00:16, 99.44it/s]

 13%|█▎        | 241/1875 [00:02<00:14, 111.67it/s]

 14%|█▎        | 257/18

 99%|█████████▉| 1864/1875 [00:15<00:00, 115.82it/s]

100%|██████████| 1875/1875 [00:15<00:00, 117.91it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

  1%|          | 10/1875 [00:00<00:19, 95.49it/s]

Epoch: 0; Accuracy train: 89.590%




  1%|          | 23/1875 [00:00<00:18, 102.47it/s]

  2%|▏         | 33/1875 [00:00<00:18, 101.23it/s]

  2%|▏         | 46/1875 [00:00<00:17, 106.86it/s]

  3%|▎         | 58/1875 [00:00<00:16, 109.52it/s]

  4%|▍         | 73/1875 [00:00<00:15, 117.71it/s]

  5%|▍         | 87/1875 [00:00<00:14, 121.70it/s]

  5%|▌         | 100/1875 [00:00<00:14, 123.54it/s]

  6%|▌         | 112/1875 [00:00<00:15, 111.54it/s]

  7%|▋         | 124/1875 [00:01<00:15, 109.70it/s]

  7%|▋         | 138/1875 [00:01<00:14, 116.77it/s]

  8%|▊         | 152/1875 [00:01<00:14, 122.62it/s]

  9%|▉         | 166/1875 [00:01<00:13, 126.45it/s]

 10%|▉         | 180/1875 [00:01<00:13, 129.57it/s]

 10%|█         | 194/1875 [00:01<00:13, 123.82it/s]

 11%|█         | 207/1875 [00:01<00:13, 122.29it/s]

 12%|█▏        | 220/1875 [00:01<00:13, 120.45it/s]

 12%|█▏        | 233/1875 [00:01<00:13, 119.97it/s]

 13%|█▎        | 246/1875 [00:02<00:14, 109.95it/s]

 14%|█▍        | 258/1875 [00:02<00:15, 103.34it/s

 96%|█████████▋| 1809/1875 [00:15<00:00, 100.24it/s]

 97%|█████████▋| 1822/1875 [00:15<00:00, 106.15it/s]

 98%|█████████▊| 1833/1875 [00:15<00:00, 103.09it/s]

 98%|█████████▊| 1844/1875 [00:16<00:00, 99.13it/s] 

 99%|█████████▉| 1858/1875 [00:16<00:00, 108.43it/s]

100%|█████████▉| 1872/1875 [00:16<00:00, 116.18it/s]

100%|██████████| 1875/1875 [00:16<00:00, 115.04it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

  1%|          | 14/1875 [00:00<00:13, 139.63it/s]

...

Epoch: 7; Accuracy train: 98.622%




  2%|▏         | 31/1875 [00:00<00:12, 150.80it/s]

  2%|▏         | 44/1875 [00:00<00:12, 142.23it/s]

  3%|▎         | 58/1875 [00:00<00:12, 140.42it/s]

  4%|▍         | 73/1875 [00:00<00:12, 142.54it/s]

  5%|▍         | 89/1875 [00:00<00:12, 145.39it/s]

  6%|▌         | 104/1875 [00:00<00:12, 145.40it/s]

  6%|▋         | 120/1875 [00:00<00:11, 147.69it/s]

  7%|▋         | 136/1875 [00:00<00:11, 149.74it/s]

  8%|▊         | 152/1875 [00:01<00:11, 152.15it/s]

  9%|▉         | 168/1875 [00:01<00:11, 154.19it/s]

 10%|▉         | 184/1875 [00:01<00:10, 153.86it/s]

 11%|█         | 200/1875 [00:01<00:10, 155.36it/s]

 12%|█▏        | 216/1875 [00:01<00:10, 155.10it/s]

 12%|█▏        | 232/1875 [00:01<00:10, 155.91it/s]

 13%|█▎        | 248/1875 [00:01<00:10, 156.06it/s]

 14%|█▍        | 264/1875 [00:01<00:10, 153.43it/s]

 15%|█▍        | 281/1875 [00:01<00:10, 156.32it/s]

 16%|█▌        | 297/1875 [00:01<00:10, 156.34it/s]

 17%|█▋        | 313/1875 [00:02<00:09, 156.21it/

Epoch: 8; Accuracy train: 98.765%




  2%|▏         | 29/1875 [00:00<00:13, 138.70it/s]

  2%|▏         | 44/1875 [00:00<00:13, 139.81it/s]

  3%|▎         | 58/1875 [00:00<00:13, 138.26it/s]

  4%|▍         | 74/1875 [00:00<00:12, 143.13it/s]

  5%|▍         | 89/1875 [00:00<00:12, 144.24it/s]

  5%|▌         | 103/1875 [00:00<00:12, 141.88it/s]

  6%|▋         | 118/1875 [00:00<00:12, 142.85it/s]

  7%|▋         | 134/1875 [00:00<00:11, 145.87it/s]

  8%|▊         | 149/1875 [00:01<00:11, 145.74it/s]

  9%|▊         | 164/1875 [00:01<00:11, 145.89it/s]

 10%|▉         | 180/1875 [00:01<00:11, 148.60it/s]

 10%|█         | 195/1875 [00:01<00:11, 147.10it/s]

 11%|█▏        | 211/1875 [00:01<00:11, 148.67it/s]

 12%|█▏        | 227/1875 [00:01<00:10, 149.96it/s]

 13%|█▎        | 243/1875 [00:01<00:10, 151.33it/s]

 14%|█▍        | 259/1875 [00:01<00:10, 151.58it/s]

 15%|█▍        | 275/1875 [00:01<00:10, 150.64it/s]

 16%|█▌        | 291/1875 [00:01<00:10, 151.15it/s]

 16%|█▋        | 307/1875 [00:02<00:10, 150.70it/

Epoch: 9; Accuracy train: 98.935%


**Другие оптимизаторы:** https://pytorch.org/docs/stable/optim.html

## 4. Зачем нужен Deep Learning ("Глубокое обучение") и почему он появился?

См. презентацию в файле DL.pdf.

## 5. Пример с MNIST. Сверточная сеть.

Идея: обработка участка изображения должна проходить независимо от конкретного расположения участка.

<img src='conv.png'>

Визуализация применения фильтров на модельном примере: http://cs231n.github.io/convolutional-networks/

Фильтры: http://cs231n.github.io/understanding-cnn/

Популярные архитектуры сверточных сетей: https://github.com/pytorch/vision/tree/master/torchvision/models

## 6. Прочие примеры DL успехов.

**Автокодировщик**

<img src='AutoEncoder.png'>
<img src='autoencoder_schema.jpg'>

Идея:  
Выделить признаки и закономерности, характеризующие данные. Можно сделать это в пространстве меньшей размерности.  
Сделаем так, чтобы сеть обучалась на некоторых данных выдавать те же самые данные. Но с ограничением, слой автокодировщика должен быть меньше (или больше), чем размерность исходных данных.

Применение автокодировщиков:
* понижение размерности (в нейронной сети нет исходных ограничений в модели, решение более универсальное)
* подавление шума (на входе на изображение добавить шум, на выходе ждать исходное изображение)
* генерация данных (variational autoencoder, нужно в слой кодировщика добавить ограничение - задать распределение)
* другие

https://vdumoulin.github.io/morphing_faces/online_demo.html

**Рекуррентные нейронные сети**

В рекуррентных нейронных сетях связи между нейронами могут идти не только от нижнего слоя к верхнему, но и от нейрона к "самому себе", точнее, к предыдущему значению самого этого нейрона или других нейронов того же слоя.

Пример: генерация текста

## Литература

1. Николенко, Кадурин, Архангельская. Глубокое обучение. Погружение в мир нейронных сетей.
2. Aurélien Géron Hands-on Machine Learning with Scikit-Learn and TensorFlow
3. Гудфеллоу, Бенджио, Курвилль. Глубокое обучение
4. Стэнфордский курс cs231n: http://cs231n.stanford.edu/